# Tau/Map2 Analysis

In [4]:
'''

Purpose of code: organize individual neuron files, analyze neurites (axons and dendrites), and run Sholl analysis.

Directory setup: main folder containing directories "data" and "code"
   --> All data files contained within "data"
   
       tif file of neuron: "name_neuron_#"
       ijroi file: "#.roi"
       simple neurite tracer swc files: "___.swc" located in directory "axon" or "dendrites"
       screenshots: taken for record keeping and stored in appropriate folders
   
   --> Code written and stored in "code"
   
Conventions: image analysis (row, column)

'''

# import required modules

import glob
import os, sys
import shutil
import cv2 
import ijroi
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# make a directory in which to direct all data output

os.mkdir('../output') 

# change directory to folder containing raw data

os.chdir('../data')

# the following conversion factor is specific to a VS-120 microscope 40X objective

p2m = 0.16094 #pixels to micron

# Organize neuron files and make axon/dendrites folders

In [35]:
# make folders with same name as neuron file

for f1 in sorted(glob.glob('*.tif')):  
# make a directory with same naming convention as neuron
    name,ext = f1.split('.')
    name,neuron,number = name.split('_')
    d = name+'_'+neuron+'_'+number
    os.mkdir(d)

In [36]:
# make directories in each of the new directories entitled "dendrites" and "axons"

for f1 in sorted(glob.glob('*_*')):
    d = f1 + '/dendrites'
    os.mkdir(d)
    a = f1 + '/axon'
    os.mkdir(a)

FileExistsError: [Errno 17] File exists: 'blue_neuron_1/dendrites'

In [37]:
# use ijmacro to draw soma and neurites (dendrites and axon), save as roi and swc files, respectively
# move original tif files into the corresponding pre-made folders

for f1 in sorted(glob.glob('*.tif')):
    name,ext = f1.split('.')
    name,neuron,number = name.split('_')
    f2 = name+'_'+neuron+'_'+number+'/'+f1
    os.rename(f1,f2)

# Load roi and extract soma size

In [13]:
# load somas and extract soma size

soma_size = {} #create a dictionary to store the information

neuron_folders = sorted(glob.glob('*_neuron_*'))
for neuron_folder in neuron_folders:
    tif_file = glob.glob(neuron_folder+'/*.tif')[0]
    img  = cv2.imread(tif_file)
    size = img.shape[:2]
    roi_file = glob.glob(neuron_folder+'/*.roi')[0]
    with open(roi_file,'rb') as f:
        roi = ijroi.read_roi(f).T
        cont = roi[::-1].T.reshape(-1,1,2).astype(np.int32)
        img2 = np.zeros(size,dtype=np.int32)
        cv2.drawContours(img2,[cont],-1,1,thickness=-1)
        roi2 = np.nonzero(img2)
        soma_area = img[roi2]
        soma_size[neuron_folder] = { 'some_size_micron': soma_area.shape[0]*(p2m**2),
                }

# Run sholl analysis

In [7]:
# define radii for sholl analysis

radii = np.arange(10,301,10)/p2m # convert to pixels

# load somas and run sholl analysis on all neurons

# create master lists containing final data
all_sholl_dict = {'radius':radii}
all_intersections = []

neuron_folders = sorted(glob.glob('*_neuron_*'))
for neuron_folder in neuron_folders:
    
    # load tif file and get shape
    tif_file = glob.glob(neuron_folder + '/*tif')[0]
    img  = cv2.imread(tif_file)
    size = img.shape[:2]
    
    # load roi file and get center
    roi_file = glob.glob(neuron_folder + '/*roi')[0]
    with open (roi_file, 'rb') as f:
        roi = ijroi.read_roi(f).T
        cont = roi[::-1].T.reshape(-1,1,2).astype(np.int32)
        img2 = np.zeros(size,dtype=np.int32)
        cv2.drawContours(img2,[cont],-1,1,thickness=-1)
        roi2 = np.nonzero(img2)
        center = np.mean(roi2,axis=1)
        
        # load swc files
        # create a list [neurites] containing coordinates of all neurites (row, column)
        # create a list [distances] containing computed distances of all neurite coordinates from the center
        swc_files = sorted(glob.glob(neuron_folder + '/dendrites/*swc'))
        dendrites = []
        distances = []
        for swc_file in swc_files:
            data = np.loadtxt(swc_file,usecols=(3,2))/p2m
            dendrites.append(data)
            distance = np.sqrt((data[:,1]-center[1])**2+(data[:,0]-center[0])**2)
            distances.append(distance)
            
            # create a list [intersections] of intersections (row, column)
            # create a list [sholl] of # of intersections 
            sholl = []
            intersections = []
            
        for r in radii:
            counter = 0
            for i in range(len(distances)):
                for j in range(len(distances[i])-1):
                    if distances[i][j] < r < distances[i][j+1] or distances[i][j] > r > distances[i][j+1]:
                        counter += 1
                        intersections.append(dendrites[i][j])
            sholl.append(counter)
        all_sholl_dict[neuron_folder] = sholl
        intersections = np.array(intersections)   
        all_intersections.append(intersections)

In [19]:
# code to export soma size and sholl data to excel sheet

df = pd.DataFrame(soma_size)
ds = pd.DataFrame(all_sholl_dict)

writer = pd.ExcelWriter('../output/sholl.xlsx') 

df.to_excel(writer,sheet_name='soma_size')
worksheet = writer.sheets['soma_size']
for c in worksheet.columns:
    worksheet.column_dimensions[c[0].column_letter].width = 15 # adjust the width of the columns

ds.to_excel(writer,sheet_name='sholl_data')
worksheet = writer.sheets['sholl_data']
for c in worksheet.columns:
    worksheet.column_dimensions[c[0].column_letter].width = 15 # adjust the width of the columns

writer.save()